### To Do / License

- Create a Marketing campaign Video idea
- Need to make sure the idea checks off 75% of the ABCD detector tests
- The video will be used for running through the ABCD detector
- Generate many 6 second Veo videos
- Combine them into a single video
- Pass the video to Gemini and ask it to write a script
- Pass the script to text to speech
- Combine the speech with the video (can we do in Python?  Camtasia)


```
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

Author: Adam Paternostro

### Pip installs

In [ ]:
# To read/write to/from Kafka
import sys

# https://pypi.org/project/moviepy/
!{sys.executable} -m pip install moviepy

### Initialize

In [ ]:
from PIL import Image
from IPython.display import HTML
from IPython.display import Audio
from functools import reduce
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

In [ ]:
# Set these (run this cell to verify the output)

bigquery_location = "us"
region = "us-central1"
location = "us-central1"
storage_account = "data-analytics-preview"
video_number_of_segments = 10
video_segment_length = 6
video_length_in_seconds = video_number_of_segments * video_segment_length
public_storage_storage_account = "data-analytics-golden-demo"
use_precanned_demo  = True
which_precanned_demo = "precanned_01_mocha_magic_ad_script"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### Helper Methods

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### Gemini LLM (Pro 1.0 , Pro 1.5)

In [ ]:
def GeminiLLM(prompt, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Gemini LLM - Multimodal

In [ ]:
def GeminiLLM_Multimodal(multimodal_prompt_list, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": multimodal_prompt_list
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{multimodal_prompt_list[0]}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Veo Image Generation

In [ ]:
def VeoVideoGen(prompt, storage_uri):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  model_version = "veo-001-preview"

  # https://cloud.google.com/vertex-ai/generative-ai/docs/image/text-to-live-images#rest
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/publishers/google/models/{model_version}:predictLongRunning"


  payload = {
    "instances": [
      {
        "prompt": prompt
      }
    ],
    "parameters": {
      "storageUri" : storage_uri
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    print(f"Video submitted for processing.")
    return True
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

#### Text to Speech Generation

In [ ]:
def TextToSpeechLanguageList(language_code):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token,
      "x-goog-user-project" : project
  }

  # https://cloud.google.com/text-to-speech/docs/reference/rest/v1/voices/list
  url = f"https://texttospeech.googleapis.com/v1/voices?languageCode={language_code}"

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    return response.text
  else:
    error = f"Error with language_code:'{language_code}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

In [ ]:
def TextToSpeech(local_filename, text, language_code, language_code_name, ssml_gender, speaking_rate = 1):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token,
      "x-goog-user-project" : project
  }

  # https://cloud.google.com/text-to-speech/docs/reference/rest/v1/text/synthesize
  url = f"https://texttospeech.googleapis.com/v1/text:synthesize"

  payload = {
   "input": {
      "text": text
   },
   "voice": {
      "languageCode": language_code,
      "name": language_code_name,
      "ssmlGender": ssml_gender # FEMALE | MALE
   },
   "audioConfig": {
      "audioEncoding": "MP3",
      "speakingRate": speaking_rate,
   }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    audio_data = json.loads(response.content)["audioContent"]
    audio_data = base64.b64decode(audio_data)
    with open(local_filename, "wb") as f:
      f.write(audio_data)
    print(f"Audio generated OK.")
    return local_filename
  else:
    error = f"Error with text:'{text}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

#### Overlay text to speech to video

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

def MergeVideoAndAudio(video_filename, audio_filename, output_filename):
  # Load the video and audio files
  video = VideoFileClip(video_filename)
  audio = AudioFileClip(audio_filename)

  # Combine the video and audio
  final_clip = video.set_audio(audio)

  # Save the combined video
  final_clip.write_videofile(output_filename)

#### Combine Videos

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

def merge_videos_sorted(folder_path, output_video_name):
  """
  Merges all MP4 video files in the specified folder into a single video,
  sorted by file name.

  Args:
      folder_path: The path to the folder containing the videos.
  """

  video_files = [f for f in os.listdir(folder_path) if f.endswith('.mp4')]
  video_files.sort()  # Sort the files by name

  clips = [VideoFileClip(os.path.join(folder_path, video)) for video in video_files]

  final_clip = concatenate_videoclips(clips)
  final_clip.write_videofile(os.path.join(folder_path, output_video_name))

#### Helper Functions

In [ ]:
def RunQuery(sql):
  import time
  from google.cloud import bigquery
  client = bigquery.Client()

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      raise Exception(query_job.error_result)

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

In [ ]:
def download_from_gcs(destination_file_name, gcs_storage_bucket, object_name):
  # prompt: Write python code to download a blob from a gcs bucket.  do not use the requests method

  from google.cloud import storage
  storage_client = storage.Client()
  bucket = storage_client.bucket(gcs_storage_bucket)

  # Construct a client side representation of a blob.
  # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
  # any content from Google Cloud Storage. As we don't need additional data,
  # using `Bucket.blob` is preferred here.
  blob = bucket.blob(object_name)
  blob.download_to_filename(destination_file_name)

  print(
      "Downloaded storage object {} from bucket {} to local file {}.".format(
          object_name, gcs_storage_bucket, destination_file_name
      )
  )

In [ ]:
# prompt: python to delete a file even if it does not exist

def delete_file(filename):
  try:
    os.remove(filename)
    print(f"File '{filename}' deleted successfully.")
  except FileNotFoundError:
    print(f"File '{filename}' not found.")

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

### Brainstorming with Gemini

##### Sample Prompt


```
Write a 1-minute video script for Chocolate A.I., a fictional eco-conscious coffee company in London, launching their new "Mocha Magic" drink. The script should be engaging for coffee lovers, highlight the sustainable practices of Chocolate A.I., and follow the ABCD framework for effective video ads (Attention, Branding, Connection, Direction).

Target Audience: Coffee lovers in London
Theme: Eco-friendly, sustainable coffee, introducing the "Mocha Magic"
Length: 60 seconds (10 x 6-second segments)
Style: Upbeat, modern, and visually appealing

Output for each 6 second segment:
Visuals:
Audio: Voiceover:
Prompt:
ABCD Rules:

Here are the ABCD rules:
Feature detected: Quick Pacing:
Feature detected: Quick Pacing (First 5 seconds):
Quick Pacing (First 5 seconds):
Dynamic Start: True
Feature detected: Supers: False
Feature detected: Supers with Audio:
Feature detected: Brand Visuals:
Feature detected: Brand Visuals (First 5 seconds):
Brand Visuals (First 5 seconds):
Feature detected: Brand Mention (Speech):
Feature detected: Brand Mention (Speech) (First 5 seconds):
Brand Mention (Speech):
Brand Mention (Speech) (First 5 seconds):
Feature detected: Product Visuals:
Feature detected:
Product Visuals (First 5 seconds):
Product Visuals: True
Product Visuals (First 5 seconds):
Feature detected: Product Mention (Text):
Feature detected: Product Mention (Text) (First 5 seconds):
Product Mention (Text):
Product Mention (Text) (First 5 seconds): False
Feature detected: Product Mention (Speech):
Feature detected: Product Mention (Speech) (First 5 seconds):
Product Mention (Speech)
Product Mention (Speech) (First 5 seconds): F
Feature detected: Visible Face (First 5 seconds):
Feature detected: Visible Face (Close Up): Visible Face (First 5 seconds):
Visible Face (Close Up):
Feature detected: Presence of People (First 5 seconds): True
Presence of People:
Presence of People (First 5 seconds):
Audio Early (First 5 seconds):
Overall Pacing:
Call To Action (Speech):
Call To Action (Text)

```

##### Gemini Sample Response

Chocolate A.I. "Mocha Magic" Video Script (60 seconds)

**Segment 1 (6 seconds)**

**Visuals:**  Fast-paced montage of iconic London scenes: bustling streets, red buses, Tower Bridge, people enjoying coffee in cafes.
**Audio:** Upbeat, trendy music bed starts.
**Voiceover:** "London, you're always on the go…"
**Prompt:**  Fast-paced, vibrant visuals to grab attention.
**ABCD Rules:** Attention, Quick Pacing, Dynamic Start, Presence of People, Audio Early, Overall Pacing: Fast.

**Segment 2 (6 seconds)**

**Visuals:** Close-up of a Chocolate A.I. barista crafting a latte art design – a swirling bean transforming into a magic hat.
**Audio:**  Music transitions to a smoother, jazzier vibe.
**Voiceover:** "But you deserve a coffee break that's truly… magical."
**Prompt:**  Introduce the brand subtly through the latte art and smoother music.
**ABCD Rules:** Branding, Brand Visuals, Product Visuals, Overall Pacing: Moderate.

**Segment 3 (6 seconds)**

**Visuals:** The barista places the finished "Mocha Magic" drink on the counter. Steam rises, revealing the name “Mocha Magic” stylishly written on the cup.
**Audio:** A gentle "whoosh" sound effect as the steam reveals the name.
**Voiceover:** "Introducing... Mocha Magic."
**Prompt:**  First clear product reveal, emphasizing the name visually and aurally.
**ABCD Rules:**  Product Visuals, Product Mention (Speech), Overall Pacing: Moderate.

**Segment 4 (6 seconds)**

**Visuals:**  Close-up shots of the drink's ingredients: ethically sourced cocoa beans, creamy oat milk being poured, a sprinkle of cinnamon.
**Audio:** Upbeat music returns, but with a lighter, more natural feel.
**Voiceover:** "Ethically sourced cocoa, creamy oat milk, and a sprinkle of magic."
**Prompt:** Highlight the premium and sustainable ingredients.
**ABCD Rules:**  Product Visuals, Connection (appeal to values), Overall Pacing: Moderate.

**Segment 5 (6 seconds)**

**Visuals:** People from diverse backgrounds enjoying Mocha Magic in different settings: a park, by the Thames, inside a cozy Chocolate A.I. cafe.
**Audio:**  Music becomes more relaxed and joyful.
**Voiceover:**  "The perfect pick-me-up, guilt-free."
**Prompt:** Show the drink being enjoyed by the target audience, emphasizing the feel-good factor.
**ABCD Rules:** Connection (emotion, lifestyle), Presence of People, Overall Pacing: Moderate.

**Segment 6 (6 seconds)**

**Visuals:** Shots of Chocolate A.I.' sustainable practices: a bicycle courier delivering coffee, a barista composting coffee grounds, a close-up of a "Fair Trade Certified" label.
**Audio:**  Uplifting music continues.
**Voiceover:**  "Chocolate A.I.: Good for you, good for the planet."
**Prompt:**  Reinforce the brand's commitment to sustainability.
**ABCD Rules:** Branding, Connection (values), Overall Pacing: Moderate.

**Segment 7 (6 seconds)**

**Visuals:** A map of London highlighting Chocolate A.I. locations. A hand reaches for a Mocha Magic drink through a cafe window.
**Audio:**  Music transitions back to the upbeat, trendy vibe from the beginning.
**Voiceover:** "Find your Mocha Magic at a Chocolate A.I. near you."
**Prompt:**  Clear call to action with location information.
**ABCD Rules:**  Direction, Call to Action (Speech), Overall Pacing: Moderate.

**Segment 8 (6 seconds)**

**Visuals:** A close-up shot of a Chocolate A.I. cup with the Mocha Magic logo and the text "Try one today!" appearing below.
**Audio:** Music fades out. A final, satisfying "slurp" sound effect.
**Voiceover:** "Chocolate A.I.. Coffee with a conscience."
**Prompt:**  End with a strong brand message and another call to action.
**ABCD Rules:** Branding, Product Visuals, Call to Action (Text), Overall Pacing: Slowing Down.

**Segment 9 (6 seconds)**

**Visuals:**  The Chocolate A.I. logo on a clean background with their website address below.
**Audio:** Silence.
**Prompt:**  Provide clear branding and website information for viewers to remember.
**ABCD Rules:**  Branding, Direction, Overall Pacing: Slow.

**Segment 10 (6 seconds)**

**Visuals:**  Fade to black.
**Audio:**  Silence.
**Prompt:**  Clean and impactful ending.
**ABCD Rules:**  Overall Pacing: Slow.


##### Call Gemini to create a storyline for our Marketing Campaign

In [ ]:
def GetMenuItems():
  sql = f"""SELECT TO_JSON(STRUCT(menu_name, menu_description)) AS menu_item_json
  FROM `chocolate_ai.menu`"""

  result_df = RunQuery(sql)
  result_str = ""

  for index, row in result_df.iterrows():
    result_str = result_str + row['menu_item_json'] + ","

  return '[' + result_str + ']'

In [ ]:
# We need to tell the LLM how to write text-to-video prompts

text_to_video_prompt_guide = """
Text-to-Video Prompt Writing Help:
<text-to-video-prompt-guide>
Here are some our best practices for text-to-video prompts:

Detailed prompts = better videos:
  - More details you add, the more control you’ll have over the video. A prompt should look like this:
  - Example Prompt: "Camera dollies to show a close up of a desperate man in a green trench coat is making a call on a rotary style wall-phone, green neon light, movie scene."
    - Here is a break down of a text-to-video prompt:
    - Camera dollies to show = Camera motion
    - A close up of = Composition
    - A desperate man in a green trench coat = Subject
    - Is making a call = Action
    - On a roary style wall-phone = Scene
    - Green Neon light = Ambiance
    - Movie Scene = Style

Use the right keywords for better control:
  - We’ve identified a list of some keywords that work well with Veo, use these in your human written prompts to get the desired camera motion or style.
  - Subject: Who or what is the main focus of the shot e.g. happy woman in her 30s
  - Scene: Where is the location of the shot (on a busy street, in space)
  - Action: What is the subject doing (walking, running, turning head)
  - Camera Motion: What the camera is doing e.g. POV shot, Aerial View, Tracking Drone view, Tracking Shot
    - Example Camera Motion: "Tracking drone view of a man driving a red convertible car in Palm Springs, 1970s, warm sunlight, long shadows"
    - Example Camera Motion: "A POV shot from a vintage car driving in the rain, Canada at night, cinematic"


Styles:
   - Overall aesthetic. Consider using specific film style keywords e.g. horror film, film noir or animated styles e.g. 3D cartoon style render
  - Example Prompt: "Over the shoulder of a young woman in a car, 1970s, film grain, horror film, cinematic he Film noir style, man and woman walk on the street, mystery, cinematic, black and white"
  - Example Prompt: "A cute creatures with snow leopard-like fur is walking in winter forest, 3D cartoon style render. An architectural rendering of a white concrete apartment building with flowing organic shapes, seamlessly blending with lush greenery and futuristic elements."

Composition:
  - How the shot is framed. This is often relative to the subject e.g. wide shot, close-up, low angle
  - Example Prompt: "Extreme close-up of a an eye with city reflected in it. A wide shot of surfer walking on a beach with a surfboard, beautiful sunset, cinematic"

Ambiance & Emotions:
  - How the color and light contribute to the scene (blue tones, night)
  - Example Prompt: "A close-up of a girl holding adorable golden retriever puppy in the park, sunlight Cinematic close-up shot of a sad woman riding a bus in the rain, cool blue tones, sad mood"

Cinematic effects:
  - e.g. double exposure, projected, glitch camera effect
  - Example Prompt: "A double exposure of silhouetted profile of a woman walking and lake, walking in a forest Close-up shot of a model with blue light with geometric shapes projected on her face"
  - Example Prompt: "Silhouette of a man walking in collage of cityscapes Glitch camera effect, close up of woman’s face speaking, neon colors"
</text-to-video-prompt-guide>
"""

In [ ]:
# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "marketing-campaign-overview" : :"text",
#    "segments" : [
#                 {
#                   "segment-number" : 0,
#                   "visuals" : "",
#                   "voiceover" : "",
#                   "video-prompt" : "",
#                   "ABCD-rules" : ["",""],
#                 }
#                 ]
#  }
response_schema = {
  "type": "object",
  "required": [
    "marketing-campaign-overview",
    "segments"
  ],
  "properties": {
    "marketing-campaign-overview": {
      "type": "string"
    },
    "segments": {
      "type": "array",
      "items": {
        "type": "object",
        "required": [
          "segement-number"
          "visuals",
          "audio",
          "voiceover",
          "ABCD-rules"
        ],
        "properties": {
          "segement-number": {
            "type": "integer",
            "format": "int64"
          },
          "visuals": {
            "type": "string"
          },
          "voiceover": {
            "type": "string"
          },
          "video-prompt": {
            "type": "string"
          },
          "ABCD-rules": {
            "type": "array",
            "items": {
              "type": "string"
            }
          }
        }
      }
    }
  }
}

menu_items = GetMenuItems()

gemini_ad_prompt = f"""Write a {video_length_in_seconds}-second video script that showcases the artistry of chocolatiers crafting exquisite desserts.
This is a new company Chocolate AI who wants to build their brand awarness.
The company is based in Paris, France.
The video will be generated by GenAI and needs to be magicical and spark the imagination of the viewers.
The video should show AI in action like data points swirling, algorithms working, personalized recommendations being generated.
Feature a range of signature offerings from the below menu.
Encourage unconventional ideas and fresh perspectives and inspires unique variations when viewing the video.
The video should be shot inside the cholcolate shop or show the street outside the shop.

Output Fields:
- Segment Number: The number of the segment.  There should be 10 segments.
- Visuals: A brief description of the visual elements for the segment.
- Voiceover: A concise voiceover script for the segment.  This should be 6 to 10 words.
- Video Prompt:
  - Read the below "Text-to-Video Prompt Writing Help" to learn more about how to create good text-to-video prompts.
  - A detailed prompt for generating the video using text-to-video technology.
  - Focus on text-to-video that shows the chocolates being made since text-to-video can generate that well.
  - Keep the prompt focused on a single thought and keep it simple.
  - Do not create a prompt that is too abstract, stick to things GenAI can easily understand.
  - Avoid using "text overlays" in the video prompt.
  - Focus on chocolates and coffee drinks in the menu
  - Feel free to use robots making the chocolates and coffee drinks, we want some text in some of the segments.
- ABCD Rules: The relevant ABCD rules that should be applied to the segment.
- Note: The ABCD rules are a specific set of guidelines that you may or may not be using.

{text_to_video_prompt_guide}

<ai-sweets-menu>
{menu_items}
</ai-sweets-menu>

<ABCD-Rules>
Quick Pacing
Quick Pacing (First 5 seconds)
Dynamic Start
Supers
Supers with Audio
Brand Visuals
Brand Visuals (First 5 seconds)
Brand Mention (Speech)
Brand Mention (Speech) (First 5 seconds)
Product Visuals
Product Visuals (First 5 seconds)
Product Mention (Text)
Product Mention (Text) (First 5 seconds)
Product Mention (Speech)
Product Mention (Speech) (First 5 seconds)
Visible Face (First 5 seconds)
Visible Face (Close Up)
Presence of People
Presence of People (First 5 seconds)
Overall Pacing
Audio Speech Early
Call To Action (Text)
Call To Action (Speech)
</ABCD-Rules>
"""


multimodal_prompt_list = [
    { "text": gemini_ad_prompt },
  #  { "fileData": {  "mimeType": "application/pdf", "fileUri": f"gs://{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Video-Text2Video/text-to-video-{formatted_date}/VeoPromptWritingGuide.pdf" } }
  ]

print(gemini_ad_prompt)
llm_result = GeminiLLM_Multimodal(multimodal_prompt_list, response_schema=response_schema)
gemini_ad_results_dict = json.loads(llm_result)
orginal_ad_results_dict = gemini_ad_results_dict # in case we swap it for a pre-canned one
print()
print(gemini_ad_results_dict)

In [ ]:
PrettyPrintJson(json.dumps(gemini_ad_results_dict))

### Create local working directories

In [ ]:
# Create a directory to download the videos from GCS and a directory to combine the text to speech and videos
directory_text_to_video = f"text-to-video-{formatted_date}"
directory_text_to_speech = f"text-to-speech-{formatted_date}"
directory_video_and_audio = f"video-and-audio-{formatted_date}"
directory_full_video = f"full-video-{formatted_date}"

os.makedirs(directory_text_to_video, exist_ok=True)
directory_text_to_video_path = os.getcwd() + f"/{directory_text_to_video}/"
print(f"directory_text_to_video_path: {directory_text_to_video_path}")

os.makedirs(directory_text_to_speech, exist_ok=True)
directory_text_to_speech_path = os.getcwd() + f"/{directory_text_to_speech}/"
print(f"directory_text_to_speech_path: {directory_text_to_speech_path}")

os.makedirs(directory_video_and_audio, exist_ok=True)
directory_video_and_audio_path = os.getcwd() + f"/{directory_video_and_audio}/"
print(f"directory_video_and_audio_path: {directory_video_and_audio_path}")

os.makedirs(directory_full_video, exist_ok=True)
directory_full_video_path = os.getcwd() + f"/{directory_full_video}/"
print(f"directory_full_video_path: {directory_full_video_path}")

full_video_filename = directory_full_video_path + "full-video.mp4"
print(f"full_video_filename: {full_video_filename}")

### Video Generation

##### Precanned Video #1

In [ ]:
# Precanned Video #1
precanned_01_mocha_magic_ad_script = {
    "marketing-campaign-overview": 'This 60-second video ad introduces London coffee lovers to "Mocha Magic," a new drink by Chocolate A.I., highlighting the company\'s commitment to sustainability and ethical sourcing.',
    "segments": [
        {
            "ABCD-rules": [
                "Quick Pacing",
                "Quick Pacing (First 5 seconds)",
                "Dynamic Start",
                "Supers with Audio",
                "Brand Visuals",
                "Brand Visuals (First 5 seconds)",
            ],
            "voiceover": "Ever wished for a coffee that's truly... magical?",
            "segement-number": 1,
            "video-prompt": "Close-up shot of a single coffee bean, rotating slowly against a black background.",
            "visuals": 'Close-up shot of a single coffee bean, rotating slowly against a black background.',
        },
        {
            "ABCD-rules": [
                "Quick Pacing",
                "Product Visuals",
                "Product Visuals (First 5 seconds)",
                "Product Mention (Text)",
                "Product Mention (Text) (First 5 seconds)",
            ],
            "voiceover": "Introducing Chocolate A.I. Mocha Magic. Crafted with care, from bean to cup.",
            "segement-number": 2,
            "video-prompt": "Aerial drone shot of vast, vibrant green coffee fields bathed in sunlight.",
            "visuals": 'Aerial drone shot of vast, vibrant green coffee fields bathed in sunlight.',
        },
        {
            "ABCD-rules": [
                "Product Visuals",
                "Visible Face (First 5 seconds)",
                "Presence of People",
                "Presence of People (First 5 seconds)",
            ],
            "voiceover": "Expertly selected beans, bursting with rich, chocolatey notes.",
            "segement-number": 3,
            "video-prompt": 'Close-up of weathered hands gently picking ripe coffee cherries.',
            "visuals": 'Close-up of weathered hands gently picking ripe coffee cherries.',
        },
        {
            "ABCD-rules": [
                "Visible Face (Close Up)",
                "Presence of People",
                "Overall Pacing",
                "Audio Speech Early",
            ],
            "voiceover": "Naturally dried under the sun, for a smooth, decadent flavor.",
            "segement-number": 4,
            "video-prompt": 'Time-lapse of coffee beans drying naturally on raised beds under the sun.',
            "visuals": 'Time-lapse of coffee beans drying naturally on raised beds under the sun.',
        },
        {
            "ABCD-rules": [
                "Brand Mention (Speech)",
                "Brand Mention (Speech) (First 5 seconds)",
                "Product Visuals",
            ],
            "voiceover": "Roasted to perfection, unlocking the magic within.",
            "segement-number": 5,
            "video-prompt": "Artisan roaster carefully monitoring coffee beans as they roast in a traditional drum roaster.",
            "visuals": "Artisan roaster carefully monitoring coffee beans as they roast in a traditional drum roaster.",
        },
        {
            "ABCD-rules": ["Product Mention (Speech)"],
            "voiceover": "Find Mocha Magic at a Chocolate A.I. truck near you!",
            "segement-number": 6,
            "video-prompt": "A sleek, modern Chocolate A.I. mobile coffee truck pulls up to a bustling London location, like Covent Garden.",
            "visuals": "A sleek, modern Chocolate A.I. mobile coffee truck pulls up to a bustling London location, like Covent Garden.",
        },
        {
            "ABCD-rules": ["Product Mention (Speech)"],
            "voiceover": "Indulge in layers of velvety chocolate, rich espresso, and a touch of sweetness.",
            "segement-number": 7,
            "video-prompt": "Barista expertly prepares a Mocha Magic latte, showcasing latte art - perhaps a swirling, magical design.",
            "visuals": "Barista expertly prepares a Mocha Magic latte, showcasing latte art - perhaps a swirling, magical design.",
        },
        {
            "ABCD-rules": ["Product Mention (Speech)"],
            "voiceover": "Mocha Magic - your new London obsession.",
            "segement-number": 8,
            "video-prompt": 'Montage of people from diverse backgrounds enjoying Mocha Magic in different London settings: by the Thames, in a park, at a market.',
            "visuals": 'Montage of people from diverse backgrounds enjoying Mocha Magic in different London settings: by the Thames, in a park, at a market.',
        },
        {
            "ABCD-rules": [
                "Product Mention (Speech)",
                "Product Mention (Speech) (First 5 seconds)",
            ],
            "voiceover": "Experience the magic. Chocolate A.I.: Sustainably Delicious.",
            "segement-number": 9,
            "video-prompt": 'Close-up of a Chocolate A.I. Mocha Magic latte with latte art, steam rising enticingly. Chocolate A.I. logo appears subtly on the cup.',
            "visuals": 'Close-up of a Chocolate A.I. Mocha Magic latte with latte art, steam rising enticingly. Chocolate A.I. logo appears subtly on the cup.',
        },
        {
            "ABCD-rules": ["Call To Action (Text)", "Call To Action (Speech)"],
            "voiceover": "Chocolate A.I.. Find Your Magic",
            "segement-number": 10,
            "video-prompt": "Start with a close-up shot of a steaming 'Mocha Magic' latte with the intricate latte art. As the steam rises and swirls, transition to a shot of a bustling London street scene with people walking by (Covent Garden or a similar vibrant location). Briefly overlay text near the end: 'Sustainably Delicious'.",
            "visuals": "Start with a close-up shot of a steaming 'Mocha Magic' latte with the intricate latte art. As the steam rises and swirls, transition to a shot of a bustling London street scene with people walking by (Covent Garden or a similar vibrant location). Briefly overlay text near the end: 'Sustainably Delicious'.",
        },
    ],
}


##### generateVideo

In [ ]:
# NOTE: This has NOT BEEN tested, it requires allow listing

def generateVideo(prompt, full_output_gcs_path_and_filename):
  return True

  """Calls Veo to create the video and waits for the output (which can be several minutes)"""

  # Call Veo and wait for the files to be generated
  # https://cloud.google.com/vertex-ai/generative-ai/docs/image/text-to-live-images#rest

  #curl -X POST \
  #    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  #    -H "Content-Type: application/json; charset=utf-8" \
  #    "https://us-central1-aiplatform.googleapis.com/v1/projects/data-analytics-preview/locations/us-central1/publishers/google/models/veo-001-preview:predictLongRunning" \
  #    -d ''

  #curl -X GET \
  #    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  #    -H "Content-Type: application/json; charset=utf-8" \
  #    "https://us-central1-aiplatform.googleapis.com/v1/projects/data-analytics-preview/locations/us-central1/publishers/google/models/veo-001-preview:fetchPredictOperation" \
  #    -d '{ "operationName": "OPERATION_NAME" }'

  model = "veo-001-preview"
  url = f"https://us-central1-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{region}/publishers/google/models/{model}:predictLongRunning"

  request_body = {
      "instances": [
          {
              "prompt": prompt
          }
        ],
      "parameters": {
          "storageUri": full_output_gcs_path_and_filename,
          "aspectRatio":"16:9"
          }
      }

  """
  ???
  """

  json_result = restAPIHelper(url, "POST", request_body)
  operation_name = json_result["operation_name"]

  url = f"https://us-central1-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{region}/publishers/google/models/{model}:fetchPredictOperation"

  request_body = {
      "operationName": operation_name
      }

  json_result = restAPIHelper(url, "POST", request_body)
  status = json_result["status"]

  while status == "RUNNING":
    time.sleep(10)
    json_result = restAPIHelper(url, "POST", request_body)
    status = json_result["status"]

  return True

##### Create the Videos (using Veo)

In [ ]:
# Veo

# Output of video files
veo_file_name = "mocha-magic-part-video-only-SEGMENT.mp4"
veo_gcs_path = f"gs://{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Video-Text2Video/veo-output/mocha-magic-ad-{formatted_date}/"
veo_bucket = storage_account

# If you want to use the pre-generate video this will use data from the public storage account
if use_precanned_demo:
  if which_precanned_demo == "precanned_01_mocha_magic_ad_script":
    # Make the precanned_01_mocha_magic_ad_script the gemini_ad_results_dict
    gemini_ad_results_dict = precanned_01_mocha_magic_ad_script
    veo_gcs_path = f"gs://{public_storage_storage_account}/chocolate-ai/v2/DB-GMA-Campaign-Assets-Video-Text2Video/Mocha-Magic/"
    veo_bucket = public_storage_storage_account

# This will take ~3-4 minutes per video (so this can take over an half an hour)
for item in gemini_ad_results_dict['segments']:
  video_prompt = item['video-prompt']
  i = item['segement-number']
  formatted_i = i
  if i < 10:
    formatted_i = f"0{i}"

  filename = veo_file_name.replace("SEGMENT", str(formatted_i))
  full_output_gcs_path_and_filename = veo_gcs_path + filename

  print(f"Generating {i}: {video_prompt} [{full_output_gcs_path_and_filename}]")

  # Generate video (if precanned then skip to save time)
  if use_precanned_demo == False:
    generateVideo(video_prompt, full_output_gcs_path_and_filename)

  item["video-veo-gcs-filename"] = full_output_gcs_path_and_filename
  item["video-gcs-copy-filename"] = f"gs://{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Video-Text2Video/mocha-magic-ad-{formatted_date}/" + filename
  item["video-local-filename"] = directory_text_to_video_path + filename

  # Download completed video
  download_from_gcs(item["video-local-filename"], veo_bucket, full_output_gcs_path_and_filename.replace(f"gs://{veo_bucket}/",""))


### Text to Speech Generation

In [ ]:
# Print out a list of language code, select one you want
response_text = TextToSpeechLanguageList("en-gb")
response_json = json.loads(response_text)
print (response_json)
# print(PrettyPrintJson(response_text))

language_code = "en-gb"
language_code_name = "en-GB-Neural2-A"
ssml_gender = "FEMALE"

In [ ]:
# Generate the text-to-speech for each segment

# Output of video files
audio_file_name = "mocha-magic-part-audio-only-SEGMENT.mp3"

for item in gemini_ad_results_dict['segments']:
  voiceover_prompt = item['voiceover']
  i = item['segement-number']
  formatted_i = i
  if i < 10:
    formatted_i = f"0{i}"

  filename = audio_file_name.replace("SEGMENT", str(formatted_i))
  item["audio-local-filename"] = directory_text_to_speech_path + filename
  item["audio-gcs-copy-filename"] = f"gs://{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Video-Text2Video/mocha-magic-ad-{formatted_date}/" + filename

  print(f"Generating {i}: {voiceover_prompt}")

  # Text-to-Speech
  TextToSpeech(item["audio-local-filename"], voiceover_prompt, language_code, language_code_name, ssml_gender, .9)
  display(Audio(item["audio-local-filename"], autoplay=True,rate=16000))
  print()


### Combine Speech and Videos

In [ ]:
# Merge the text-to-speech with the text-to-video

# Output of video files
video_and_audio_file_name = "mocha-magic-part-audio-and-video-SEGMENT.mp4"

for item in gemini_ad_results_dict['segments']:
  i = item['segement-number']
  formatted_i = i
  if i < 10:
    formatted_i = f"0{i}"

  filename = video_and_audio_file_name.replace("SEGMENT", str(formatted_i))
  item["combined-audio-and-video-local-filename"] = directory_video_and_audio_path + filename
  item["combined-audio-and-video-gcs-copy-filename"] = f"gs://{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Video-Text2Video/mocha-magic-ad-{formatted_date}/" + filename

  print(f"Merging Segment {i}")

  # Text-to-Speech
  MergeVideoAndAudio(item["video-local-filename"], item["audio-local-filename"], item["combined-audio-and-video-local-filename"])

In [ ]:
# Show the first merged file
part_01_video_with_audio_mp4 = open(gemini_ad_results_dict['segments'][0]["combined-audio-and-video-local-filename"], 'rb').read()
part_01_video_with_audio_mp4_url = "data:video/mp4;base64," + base64.b64encode(part_01_video_with_audio_mp4).decode()

In [ ]:
# Play the video
# 16:9 aspect ratio
HTML(f"""
<video width=600 height=337 controls>
      <source src="{part_01_video_with_audio_mp4_url}" type="video/mp4">
</video>
""")

In [ ]:
# Show the second merged file
part_02_video_with_audio_mp4 = open(gemini_ad_results_dict['segments'][1]["combined-audio-and-video-local-filename"], 'rb').read()
part_02_video_with_audio_mp4_url = "data:video/mp4;base64," + base64.b64encode(part_02_video_with_audio_mp4).decode()

In [ ]:
# Play the video
# 16:9 aspect ratio
HTML(f"""
<video width=600 height=337 controls>
      <source src="{part_02_video_with_audio_mp4_url}" type="video/mp4">
</video>
""")

### Merge the videos into one video


- **Warning: This does not currently produce a well edited video.**
- The code works, but a final version was created using a video editting software.

In [ ]:
# Merge the videos (the file names are when sorted match the placement of each video in the overall video)

print("Merging videos")
print(f"directory_video_and_audio_path: {directory_video_and_audio_path}")
print(f"full_video_filename: {full_video_filename}")

merge_videos_sorted(directory_video_and_audio_path, full_video_filename)

In [ ]:
# prompt: python to play a mp4 in a jupyter notebook
mocha_magic_ad_full_video_mp4 = open(full_video_filename, 'rb').read()
mocha_magic_ad_full_video_url = "data:video/mp4;base64," + base64.b64encode(mocha_magic_ad_full_video_mp4).decode()

In [ ]:
# 16:9 aspect ratio
HTML(f"""
<p>Combined video using moviepy which needs some fine tuning</p>
<video width=600 height=337 controls>
      <source src="{mocha_magic_ad_full_video_url}" type="video/mp4">
</video>
""")

In [ ]:
print("To see the Mocha Magic Video which was edited using video editing software.")
print("Open this to see the one edited manually: https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v2/DB-GMA-Campaign-Assets-Video-Text2Video/Mocha-Magic/mocha-magic-full-ad.mp4")

### Copy Artifacts to Storage

In [ ]:
# Upload videos, audios, combined audio-video to storaege
for item in gemini_ad_results_dict['segments']:
  #print(f"Copying {item['video-local-filename']} to {item['video-gcs-copy-filename']}")
  #print(f"Copying {item['audio-local-filename']} to {item['audio-gcs-copy-filename']}")
  #print(f"Copying {item['combined-audio-and-video-local-filename']} to {item['combined-audio-and-video-gcs-copy-filename']}")
  copy_file_to_gcs(item["video-local-filename"], storage_account, item["video-gcs-copy-filename"].replace(f"gs://{storage_account}/",""))
  copy_file_to_gcs(item["audio-local-filename"], storage_account, item["audio-gcs-copy-filename"].replace(f"gs://{storage_account}/",""))
  copy_file_to_gcs(item["combined-audio-and-video-local-filename"], storage_account, item["combined-audio-and-video-gcs-copy-filename"].replace(f"gs://{storage_account}/",""))


# Final combined output video
copy_file_to_gcs(item["combined-audio-and-video-local-filename"], storage_account, f"chocolate-ai/DB-GMA-Campaign-Assets-Video-Text2Video/mocha-magic-ad-{formatted_date}/full-video.mp4")

# Save the prompt so we know how we got this data
with open("gemini_ad_prompt.txt", "w") as f:
  f.write(gemini_ad_prompt)
copy_file_to_gcs("gemini_ad_prompt.txt", storage_account, f"chocolate-ai/DB-GMA-Campaign-Assets-Video-Text2Video/mocha-magic-ad-{formatted_date}/gemini_ad_prompt.txt")
delete_file("gemini_ad_prompt.txt")

# Save the output of the prompt
with open("gemini_ad_prompt_results.txt", "w") as f:
  f.write(PrettyPrintJson(json.dumps(orginal_ad_results_dict)))
copy_file_to_gcs("gemini_ad_prompt_results.txt", storage_account, f"chocolate-ai/DB-GMA-Campaign-Assets-Video-Text2Video/mocha-magic-ad-{formatted_date}/gemini_ad_prompt_results.txt")
delete_file("gemini_ad_prompt_results.txt")

In [ ]:
# To view the bucket
print(f"https://console.cloud.google.com/storage/browser/{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Video-Text2Video/mocha-magic-ad-{formatted_date}")

### Option: Combine video only (no audio)

In [ ]:
# This is used for the DB-GMA-Campaign-Assets-Video-Create-Shorts.ipy Notebook
merge_videos_sorted(directory_text_to_video_path, "mocha-magic-video-only-combined.mp4")
copy_file_to_gcs(f"{directory_text_to_video_path}mocha-magic-video-only-combined.mp4", storage_account, f"chocolate-ai/DB-GMA-Campaign-Assets-Video-Text2Video/mocha-magic-ad-{formatted_date}/mocha-magic-video-only-combined.mp4")
delete_file(f"{directory_text_to_video_path}mocha-magic-video-only-combined.mp4")

### Clean Up

In [ ]:
user_input = input("Do you want to the files on this notebook machine (Y/n)?")
if user_input == "Y":
  import shutil
  print(f"Removing directory: {directory_text_to_video_path}")
  shutil.rmtree(directory_text_to_video_path)

  print(f"Removing directory: {directory_text_to_speech_path}")
  shutil.rmtree(directory_text_to_speech_path)

  print(f"Removing directory: {directory_video_and_audio_path}")
  shutil.rmtree(directory_video_and_audio_path)

  print(f"Removing directory: {directory_full_video_path}")
  shutil.rmtree(directory_full_video_path)

### Reference Links


- [Google.com](https://www.google.com)